In [17]:
import spikeinterface.full as si
import matplotlib.pyplot as plt
import numpy as np
import probeinterface as pi
from pathlib import Path
import pandas as pd 
import os, sys
import shutil


    
global_job_kwargs = dict(n_jobs=-1, chunk_duration="10s",progress_bar=True)
si.set_global_job_kwargs(**global_job_kwargs)


basefolder= r"D:\Data\raw\3198-51\51_naive"
base_path = Path(basefolder)





recording =  si.read_spikeglx(basefolder, stream_id='imec0.ap', load_sync_channel=False)
lfp = si.read_spikeglx(basefolder, stream_id='imec0.lf', load_sync_channel=False)
event =  si.read_spikeglx(basefolder, stream_id='nidq', load_sync_channel=False)
print(recording)


#recording = si.ChannelSliceRecording(recording, channel_ids=recording.get_channel_ids()[180:330])
rec1 = si.highpass_filter(recording, freq_min=400.)
rec1 = si.phase_shift(rec1)
bad_channel_ids, channel_labels = si.detect_bad_channels(rec1,method = 'coherence+psd')
print(bad_channel_ids)
rec1 = si.interpolate_bad_channels(recording=rec1, bad_channel_ids=bad_channel_ids)

rec1 = si.common_reference(rec1, operator="median", reference="global")
print(rec1)



SpikeGLXRecordingExtractor: 384 channels - 29999.900000 Hz - 1 segments - 137,376,461 samples 
                            4,579.23s (1.27 hours) - int16 dtype - 98.26 GiB
['imec0.ap#AP2' 'imec0.ap#AP34' 'imec0.ap#AP67' 'imec0.ap#AP116'
 'imec0.ap#AP191']
CommonReferenceRecording: 384 channels - 29999.900000 Hz - 1 segments - 137,376,461 samples 
                          4,579.23s (1.27 hours) - int16 dtype - 98.26 GiB


In [20]:
lfp = si.common_reference(lfp,operator="median", reference="global")

from spikeinterface.exporters import export_to_ibl_gui
recording = si.read_kilosort(r"D:\Data\raw\3198-51\51_naive\sorted\sorter_output")
analyzer = si.create_sorting_analyzer(recording, rec1, sparse=True, format="memory")
analyzer.compute(['random_spikes', 'waveforms', 'templates', 'spike_amplitudes','quality_metrics',],**global_job_kwargs)
output_path = base_path / 'export_to_ibl2'

#


# the export process is fast because everything is pre-computed
export_to_ibl_gui(
    sorting_analyzer=sorting_analyzer,
    output_folder= output_path,
    lfp_recording=lfp,
    n_jobs=-1
)

estimate_sparsity (workers: 16 processes):   0%|          | 0/458 [00:00<?, ?it/s]

ValueError: Extension '"quality_metrics"' is unknown maybe this is an external extension or a typo.

In [21]:
analyzer.compute(['quality_metrics'])
output_path = base_path / 'export_to_ibl2'

#


# the export process is fast because everything is pre-computed
export_to_ibl_gui(
    sorting_analyzer=analyzer,
    output_folder= output_path,
    lfp_recording=lfp,
    n_jobs=-1
)

AssertionError: Extension quality_metrics requires templates to be computed first

In [16]:
recording_lfp

%matplotlib widget
si.plot_traces({'raw':lfp,'filtered':recording_lfp}, backend='ipywidgets')



AppLayout(children=(TimeSlider(children=(Dropdown(description='segment', options=(0,), value=0), Button(icon='…